## NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

print(nlp.pipe_names)

In [ ]:
from app.templates.meat_sale.domain_model_template import meat_sale_domain_model_template as dm
from app.templates.meat_sale.contract_spec_template import meat_sale_contract_spec_template as cs
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

ob = cs.obligations['delivery']
ob.to_sym()

In [ ]:
from app.templates.meat_sale.contract_template import get_template

contract_template = get_template()

In [ ]:
test_suite = [
    ['at the warehouse', 'warehouse'], 
    ['to the warehouse', 'warehouse'], 
    ['at their warehouse', 'buyer.address'], 
    ['at its warehouse', 'buyer.address'], 
    ['at the buyer\'s address', 'buyer.address'],
    ['at 883 5th Ave, Ottawa, ON', '1234 Main Street, Ottawa, ON'],
    #...
]

index = 4
sentence = test_suite[index][0]

doc = nlp(sentence)

summarizer.summarize(sentence)

In [ ]:
from app.src.rules.meat_sale.delivery_location.location_extractor import LocationExtractor
from app.src.rules.meat_sale.delivery_location.address_extractor import AddressExtractor
from app.src.rules.meat_sale.delivery_location.matchers import get_validation_matcher

from app.src.rules.meat_sale.delivery_location.role_score_builder import RoleScoreBuilder
from app.src.rules.meat_sale.delivery_location.coref_getter import CorefGetter
from app.src.rules.shared.match_validator import MatchValidator

from app.src.rules.meat_sale.delivery_location.basic_scorer import BasicScorer
from app.src.rules.meat_sale.delivery_location.address_scorer import AddressScorer
from app.src.rules.meat_sale.delivery_location.noun_chunk_scorer import NounChunkScorer
from app.src.rules.meat_sale.delivery_location.role_scorer import RoleScorer

init_role_scores = {
    'buyer': 0.6,
    'seller': 0.4
}

key = 'DELIVERY_LOCATION'

coref_getter = CorefGetter(nlp)

address_extractor = AddressExtractor()

role_score_builder = RoleScoreBuilder(nlp, init_role_scores, coref_getter)

scorers = [
    BasicScorer('buyer', 0),
    NounChunkScorer(nlp),
    AddressScorer(nlp, address_extractor),
    RoleScorer(nlp, role_score_builder)
]

v_matcher = get_validation_matcher(nlp)
validator = MatchValidator(v_matcher)

location_extractor = LocationExtractor(nlp, validator, scorers)

In [ ]:
from app.classes.contract_update_request import ContractUpdateRequest

req = ContractUpdateRequest(
    contract_template,
    key,
    sentence,
    doc
)

result = location_extractor.extract(req)

In [ ]:
print(result)